In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config; set_config(display='diagram')

In [2]:
from merge_years.import_data import get_full_data


In [3]:
!cd ..

In [4]:
raw_data=get_full_data('../raw_data')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
raw_data.shape

In [ ]:
raw_data = raw_data[raw_data.name != 'ben davies']
print(raw_data.shape)
#raw_data = raw_data[raw_data.name != 'danny ward']
#raw_data.shape

In [ ]:
raw_data[raw_data['position'].isnull()]


In [ ]:
j=0
for i,row in raw_data.iterrows():
    if row['name']=='caglar söyüncü':
        print(j)
        raw_data['position'].at[i]='DEF'
        j+=1

In [ ]:
raw_data.isnull().sum().sort_values(ascending=False)

In [ ]:
raw_data.columns

In [ ]:
'aaron connolly' in raw_data[raw_data['season']==21]['name'].unique()

# Feature engineering

In [ ]:
raw_data.columns

In [ ]:
raw_data.head()

In [ ]:
features = ['name','assists','bps','clean_sheets','goals_scored',
            'opponent_level','transfers_balance','was_home','position',
           'saves','kickoff_time','value','dreamteam_yearly_average','GW','season','total_points','kickoff_date']
subset_df = raw_data[features]

In [ ]:
subset_df.head()

In [ ]:
time=[]
for row in subset_df['kickoff_time']:
    time.append(row.hour)
subset_df['time']=time

In [ ]:
subset_df.drop(columns='kickoff_time',inplace=True)

In [ ]:
subset_df

In [ ]:
rolling_features=['name','assists','bps','clean_sheets','goals_scored','opponent_level',
                   'transfers_balance','was_home','saves','value','total_points','time']
#missing position, dreamteam, GW, Season



In [ ]:
rolling_df=subset_df[rolling_features].groupby('name').rolling(5,closed = 'left').mean()

In [ ]:
rolling_df.head(10)

In [ ]:
rolling_df.rename(columns={'total_points':'rolling_points'},inplace=True)


result = map(lambda position:position, subset_df['position'])
rolling_df['position']=list(result)

result = map(lambda position:position, subset_df['dreamteam_yearly_average'])
rolling_df['dreamteam_yearly_average']=list(result)

result = map(lambda position:position, subset_df['GW'])
rolling_df['GW']=list(result)

result = map(lambda position:position, subset_df['season'])
rolling_df['season']=list(result)

result = map(lambda position:position, subset_df['total_points'])
rolling_df['total_points']=list(result)

result = map(lambda position:position, subset_df['kickoff_date'])
rolling_df['kickoff_date']=list(result)

In [ ]:
rolling_df.tail()


In [ ]:
len(rolling_df['assists'].dropna())

In [ ]:
not_processed_df_name=rolling_df.dropna(subset=['assists'])

In [ ]:
not_processed_df=not_processed_df_name.reset_index()

In [ ]:
player_dtya={}
for i,player in enumerate(not_processed_df['name'].unique()):
    player_dtya[player]=not_processed_df[not_processed_df['name']==player]['dreamteam_yearly_average'].max()
    if i%100==0:
        print(player,i)

In [ ]:
ls=[]
for i,row in not_processed_df.iterrows():
    ls.append(player_dtya[row['name']])
not_processed_df['dreamteam_yearly_average']=ls

In [ ]:
not_processed_df.head(350)

In [ ]:
not_processed_df.isnull().sum().sort_values(ascending=False)

In [ ]:

feature_selected_df=not_processed_df.dropna().drop(columns=['level_1','season'])#season

In [ ]:
feature_selected_df.isnull().sum().sort_values(ascending=False)

In [ ]:
feature_selected_df.shape

# Preprocessing

In [ ]:
feature_selected_df.head(550)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc=OneHotEncoder()

enc.fit(feature_selected_df[['position']])

#enc.categories_
#order for position is ['DEF','FWD','GK','MID']
positions=enc.transform(feature_selected_df[['position']]).toarray()
feature_selected_df["DEF"],feature_selected_df["FWD"],feature_selected_df['GK'], feature_selected_df['MID'] = positions.T 

feature_selected_df.head(500)

In [ ]:
final_df=feature_selected_df.drop(columns='position')

In [ ]:
final_df

# Make model

In [ ]:
X=final_df.drop(columns='total_points')
y=final_df['total_points']

In [ ]:
X.drop(columns=['name','GW','kickoff_date'],inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

reg = GradientBoostingRegressor(random_state=0,loss='ls')
reg.fit(X_train, y_train)
y_pred=reg.predict(X_test)

mean_absolute_error(y_test,y_pred)


In [ ]:
for i,value in enumerate(y_pred):
    print(y_test.iloc[i],value)


In [ ]:
plt.scatter(y_test,y_pred)

# Trying to detect 5 players last 4-5 gameweeks

In [ ]:
final_df

In [ ]:
#train_df = final_df[final_df.kickoff_date < pd.to_datetime('2021-05-07')]
#test_df = final_df[final_df.kickoff_date >= pd.to_datetime('2021-05-07')]

train_df = final_df[final_df.GW < 29]
test_df = final_df[final_df.GW >= 34]

test_df = test_df[test_df.kickoff_date > pd.to_datetime('2021-01-01')]

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
new_df=test_df[['name','total_points','GW']]

In [ ]:
X_train=train_df.drop(columns=['total_points','name','GW','kickoff_date'])
y_train=train_df['total_points']

X_test=test_df.drop(columns=['total_points','name','GW','kickoff_date'])
y_test=test_df['total_points']

In [ ]:
corrMatrix = train_df.corr()
print (corrMatrix)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train.head()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

reg = KNeighborsRegressor()
reg.fit(X_train, y_train)
y_pred=reg.predict(X_test)

mean_absolute_error(y_test,y_pred)

In [ ]:
new_df['y_pred']=y_pred

In [ ]:
new_df[new_df['name']=='heung-min son']

In [ ]:
new_df[new_df['name']=='kyle walker']

In [ ]:
new_df[new_df['name']=='jesse lingard']

In [ ]:
new_df[new_df['name']=='youri tielemans']

In [ ]:
new_df[new_df['name']=='james justin']

In [ ]:
plt.plot(new_df[new_df['name']=='youri tielemans']['GW'],np.abs(new_df[new_df['name']=='youri tielemans']['y_pred']-new_df[new_df['name']=='youri tielemans']['total_points']),label='youri tielemans')
plt.plot(new_df[new_df['name']=='timo werner']['GW'],np.abs(new_df[new_df['name']=='timo werner']['y_pred']-new_df[new_df['name']=='timo werner']['total_points']),label='timo werner')
plt.plot(new_df[new_df['name']=='jesse lingard']['GW'],np.abs(new_df[new_df['name']=='jesse lingard']['y_pred']-new_df[new_df['name']=='jesse lingard']['total_points']),label='jesse lingard')
plt.plot(new_df[new_df['name']=='james justin']['GW'],np.abs(new_df[new_df['name']=='james justin']['y_pred']-new_df[new_df['name']=='james justin']['total_points']),label='james justin')

plt.legend()



In [ ]:
player_dtya={}
for i,player in enumerate(new_df['name'].unique()):
    
    print(player,i)

# Other stuff

In [ ]:
def get_assists_per_game(df,player):
    ls=[0]
    assists=0
    num_games=0
    for i,row in df.iterrows():
        if row['minutes']>0:
            num_games+=1
            assists+=row['assists']
        if num_games>0:
            ls.append(assists/num_games)
        else:
            ls.append(0)
    del ls[-1]
    return ls[-1]

In [ ]:
def get_goals_per_game(df,player):
    ls=[0]
    goals=0
    num_games=0
    for i,row in df.iterrows():
        if row['minutes']>0:
            num_games+=1
            goals+=row['goals_scored']
        if num_games>0:
            ls.append(goals/num_games)
        else:
            ls.append(0)
    del ls[-1]
    return ls[-1]

def get_clean_sheets_per_game(df,player):
    ls=[0]
    clean_sheets=0
    num_games=0
    for i,row in df.iterrows():
        if row['minutes']>0:
            num_games+=1
            clean_sheets+=row['clean_sheets']
        if num_games>0:
            ls.append(clean_sheets/num_games)
        else:
            ls.append(0)
    del ls[-1]
    return ls[-1]

def get_minutes_per_game(df,player):
    ls=[0]
    minutes=0
    num_games=0
    for i,row in df.iterrows():
        if row['minutes']>0:
            num_games+=1
            minutes+=row['minutes']
        if num_games>0:
            ls.append(minutes/num_games)
        else:
            ls.append(0)
    del ls[-1]
    return ls[-1]

In [ ]:
X_raw.shape

In [ ]:
assists_per_game=[]
goals_per_game=[]
clean_sheets_per_game=[]
minutes_per_game=[]

for i,player in enumerate(X_raw['name'].unique()):
    if i%100==0:
        print(player,i)
    assists_per_game+=get_assists_per_game(X_raw[X_raw['name']==player],player)
    goals_per_game+=get_goals_per_game(X_raw[X_raw['name']==player],player)
    clean_sheets_per_game+=get_clean_sheets_per_game(X_raw[X_raw['name']==player],player)
    minutes_per_game+=get_minutes_per_game(X_raw[X_raw['name']==player],player)

In [ ]:
X_raw['apg']=assists_per_game
X_raw['gpg']=goals_per_game
X_raw['cspg']=clean_sheets_per_game
X_raw['mpg']=minutes_per_game

In [ ]:
X_raw.columns

In [ ]:
X_raw.shape

In [ ]:
X_raw.drop(columns=['assists','bonus','bps','clean_sheets','element','fixture','goals_conceded','goals_scored'
                    ,'minutes','opponent_team','own_goals','penalties_missed','penalties_saved','red_cards','saves'
                    ,'selected','team_a_score','team_h_score','transfers_balance','transfers_in','transfers_out'
                    ,'value','yellow_cards','team_id','team_name','ict_index','influence','threat','creativity'],inplace=True) #Bonus, bps

In [ ]:
X_raw.shape

In [ ]:
X_raw.columns

In [ ]:
X_raw.head()